## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-11-10-06-53-00 +0000,wsj,The ruling means the government will have to m...,https://www.wsj.com/politics/policy/appeals-co...
1,2025-11-10-06-48-00 +0000,wsj,U.S. Treasury Yields Rise on Prospect of End t...,https://www.wsj.com/finance/investing/jgbs-edg...
2,2025-11-10-06-39-41 +0000,nypost,Florida military families told to take down ea...,https://nypost.com/2025/11/10/us-news/florida-...
3,2025-11-10-06-22-26 +0000,nyt,Senate Moves Toward Ending Shutdown After Demo...,https://www.nytimes.com/2025/11/09/us/politics...
4,2025-11-10-06-21-22 +0000,bbc,'Jersey isn't as family-friendly as it should be',https://www.bbc.com/news/articles/c8670wpqp57o...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2316/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
51,trump,28
16,shutdown,20
143,new,15
2,government,9
107,typhoon,8


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
133,2025-11-09-18-46-21 +0000,nypost,SNAP thrown into chaos after Trump admin tells...,https://nypost.com/2025/11/09/us-news/snap-thr...,80
23,2025-11-10-05-02-42 +0000,nyt,Trump Renews Attacks on Obamacare in New Push ...,https://www.nytimes.com/2025/11/09/us/politics...,79
90,2025-11-09-22-28-52 +0000,nypost,Senate eyes vote on potential breakthrough to ...,https://nypost.com/2025/11/09/us-news/senate-e...,68
33,2025-11-10-03-56-00 +0000,wsj,The Senate late Sunday cleared a critical proc...,https://www.wsj.com/politics/policy/republican...,61
29,2025-11-10-04-14-40 +0000,nyt,Sean Duffy Juggles Shutdown Roles on Flight Sa...,https://www.nytimes.com/2025/11/09/us/politics...,60


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
133,80,2025-11-09-18-46-21 +0000,nypost,SNAP thrown into chaos after Trump admin tells...,https://nypost.com/2025/11/09/us-news/snap-thr...
33,41,2025-11-10-03-56-00 +0000,wsj,The Senate late Sunday cleared a critical proc...,https://www.wsj.com/politics/policy/republican...
156,38,2025-11-09-14-46-42 +0000,bbc,BBC on Luzon Island as super typhoon hits Phil...,https://www.bbc.com/news/videos/c20p2m83p5go?a...
18,34,2025-11-10-05-22-38 +0000,wapo,Special counsel indicts former South Korean pr...,https://www.washingtonpost.com/world/2025/11/1...
68,31,2025-11-10-00-09-50 +0000,nypost,Brooklyn dad says goodbye to hero son Itay Che...,https://nypost.com/2025/11/09/world-news/nyc-d...
149,27,2025-11-09-15-33-10 +0000,nypost,Lefty Gen-Z NYC Councilman Chi Osse gears up f...,https://nypost.com/2025/11/09/us-news/nyc-coun...
65,23,2025-11-10-00-12-41 +0000,latimes,Here's how many flights have been canceled out...,https://www.latimes.com/california/story/2025-...
148,23,2025-11-09-15-56-32 +0000,nypost,Transportation secretary gives dire warning on...,https://nypost.com/2025/11/09/us-news/transpor...
182,23,2025-11-09-10-59-00 +0000,startribune,Taylor Schlitz: Would you go on a date for fre...,https://www.startribune.com/young-people-datin...
86,22,2025-11-09-22-53-45 +0000,nyt,Flight Cancellations and Delays May Get Worse ...,https://www.nytimes.com/2025/11/09/business/ai...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
